In [1]:
# Open the file in read mode
with open('questions.txt', 'r') as file:
    # Read all lines in the file
    questions = file.readlines()

# Now you can access each question by its index, like in a list
# for i in range(len(questions)):
#     print(f"Question[{i}]: {questions[i].strip()}")

# print(questions[2])

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from datasets import DatasetDict, Dataset

# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

# articles available here:  {add GitHub repo}
documents = SimpleDirectoryReader("pdf").load_data()

# some ad hoc document refinement
for doc in documents:
    if "Member-only story" in doc.text:
        documents.remove(doc)
        continue

    if "The Data Entrepreneurs" in doc.text:
        documents.remove(doc)

    if " min read" in doc.text:
        documents.remove(doc)

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)
# set number of docs to retrieve
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

# load questions
with open('questions.txt', 'r') as file:
    questions = file.readlines()

# strip newline characters from questions
questions = [q.strip() for q in questions]

# create a list to store the mapping of questions to their respective contexts
question_context_pairs = []

# iterate over each question and query the engine
for i, question in enumerate(questions):
    response = query_engine.query(question)

    # create a combined context from the top 3 chunks
    context = "Context:\n"
    for j in range(top_k):
        context = context + response.source_nodes[j].text + "\n\n"

    # store the mapping of the question to its respective context
    question_context_pairs.append({"question": question, "context": context})

# create a dictionary with two lists: questions and contexts
data = {
    "question": [pair["question"] for pair in question_context_pairs],
    "context": [pair["context"] for pair in question_context_pairs]
}

# create a Hugging Face Dataset from the dictionary
dataset = Dataset.from_dict(data)

# print the dataset
dataset

/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM is explicitly disabled. Using MockLLM.


Dataset({
    features: ['question', 'context'],
    num_rows: 3
})

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id="mistralai/Mistral-7B-Instruct-v0.2"
# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    cache_dir = "/mnt/data1/backup/viswaz/Project_K/huggingface_cache/",
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# import re
# def generate_response(prompt, model):
#   encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
#   model_inputs = encoded_input.to('cuda')

#   generated_ids = model.generate(**model_inputs,
#                                  max_new_tokens=150,
#                                  do_sample=True,
#                                  pad_token_id=tokenizer.eos_token_id)

#   decoded_output = tokenizer.batch_decode(generated_ids)

#   # Split the generated text into prompt and answer
#   split_index = decoded_output[0].find('answer:')
#   if split_index == -1:
#     split_index = decoded_output[0].find('Answer:')
#   decoded_output = [s.strip("'") for s in decoded_output]
#   decoded_output = [s.replace('</s>', '') for s in decoded_output]
#   answer = decoded_output[0][split_index+len('answer:'):].strip()
#   answer = answer.replace('[INST]'or '[ANS]' , '').replace('[/INST]' or '[/ANS]' or '[//INST]', '')
#   answer = re.sub(r'\[.*?\]', '', answer)

#   # Stop decoding when it encounters a ### token
#   stop_index = answer.find('###')
#   if stop_index != -1:
#     answer = answer[:stop_index].strip()

#   return answer


In [7]:

# # Initialize an empty list to store the answers
# answers = []

# # Iterate over each question in the dataset
# for i in range(len(dataset)):
#     # Get the question and the context
#     question = dataset[i]["question"]
#     context = dataset[i]["context"]

#     # Define the prompt as the concatenation of the context and the question
#     prompt = prompt_template_w_context(context, question)

#     outputs = generate_response(prompt, model)

#     print(tokenizer.batch_decode(outputs)[0])

# # # Print the answers
# # for i, answer in enumerate(answers):
# #     print(f"Question {i+1}: {dataset[i]['question']}")
# #     print("\n\n\n")
# #     print(f"Answer: {answer}\n")
# #     print("\n\n\n")


In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from datasets import DatasetDict, Dataset

# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

# articles available here:  {add GitHub repo}
documents = SimpleDirectoryReader("pdf").load_data()

# some ad hoc document refinement
for doc in documents:
    if "Member-only story" in doc.text:
        documents.remove(doc)
        continue

    if "The Data Entrepreneurs" in doc.text:
        documents.remove(doc)

    if " min read" in doc.text:
        documents.remove(doc)

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)
# set number of docs to retrieve
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

# load questions
with open('questions.txt', 'r') as file:
    questions = file.readlines()

# strip newline characters from questions
questions = [q.strip() for q in questions]

# create a list to store the mapping of questions to their respective contexts
question_context_pairs = []

# iterate over each question and query the engine
for i, question in enumerate(questions):
    response = query_engine.query(question)

    # create a combined context from the top 3 chunks
    context = "Context:\n"
    for j in range(top_k):
        context = context + response.source_nodes[j].text + "\n\n"

    # store the mapping of the question to its respective context
    question_context_pairs.append({"question": question, "context": context})

# create a dictionary with two lists: questions and contexts
data = {
    "question": [pair["question"] for pair in question_context_pairs],
    "context": [pair["context"] for pair in question_context_pairs]
}

# create a Hugging Face Dataset from the dictionary
dataset = Dataset.from_dict(data)

from transformers import AutoTokenizer, AutoModelForCausalLM

model_id="mistralai/Mistral-7B-Instruct-v0.2"
# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    cache_dir = "/mnt/data1/backup/viswaz/Project_K/huggingface_cache/",
)

def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=150,
                                 do_sample=True,
                                 pad_token_id=tokenizer.eos_token_id)

  return generated_ids

# prompt (no context)
intstructions_string = f""" you are a textbot that helps in finding answers to questions in the research papers, blogs,pdf's or any text context.
,make your answers more meaningful and short,end all responses with a signature after answer "-yourbot"

please answer the following question
"""
prompt_template_w_context = lambda context, question: f'''[INST] {intstructions_string}

{context}

Please answer to the following question. Use the context above if it is helpful.

{question}

[/INST]'''
# Initialize an empty list to store the answers
answers = []

# Iterate over each question in the dataset
for i in range(len(dataset)):
    # Get the question and the context
    question = dataset[i]["question"]
    context = dataset[i]["context"]

    # Define the prompt as the concatenation of the context and the question
    prompt = prompt_template_w_context(context, question)

    outputs = generate_response(prompt, model)

    # Decode the generated IDs and store the answer
    decoded_output = tokenizer.batch_decode(outputs)[0]
    answers.append(decoded_output)

# Print the answers
for i, answer in enumerate(answers):
    print(f"Question {i+1}: {dataset[i]['question']}")
    print("\n\n\n")
    print(f"Answer: {answer}\n")
    print("\n\n\n")


/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM is explicitly disabled. Using MockLLM.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from datasets import DatasetDict, Dataset

# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

# articles available here:  {add GitHub repo}
documents = SimpleDirectoryReader("pdf").load_data()

# some ad hoc document refinement
for doc in documents:
    if "Member-only story" in doc.text:
        documents.remove(doc)
        continue

    if "The Data Entrepreneurs" in doc.text:
        documents.remove(doc)

    if " min read" in doc.text:
        documents.remove(doc)

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)
# set number of docs to retrieve
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

# load questions
with open('questions.txt', 'r') as file:
    questions = file.readlines()

# strip newline characters from questions
questions = [q.strip() for q in questions]

# create a list to store the mapping of questions to their respective contexts
question_context_pairs = []

# iterate over each question and query the engine
for i, question in enumerate(questions):
    response = query_engine.query(question)

    # create a combined context from the top 3 chunks
    context = "Context:\n"
    for j in range(top_k):
        context = context + response.source_nodes[j].text + "\n\n"

    # store the mapping of the question to its respective context
    question_context_pairs.append({"question": question, "context": context})

# create a dictionary with two lists: questions and contexts
data = {
    "question": [pair["question"] for pair in question_context_pairs],
    "context": [pair["context"] for pair in question_context_pairs]
}

# create a Hugging Face Dataset from the dictionary
dataset = Dataset.from_dict(data)


/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/data1/backup/viswaz/Project_K/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM is explicitly disabled. Using MockLLM.


In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model_id="mistralai/Mistral-7B-Instruct-v0.2"
# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    cache_dir = "/mnt/data1/backup/viswaz/Project_K/huggingface_cache/",
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:

def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=150,
                                 do_sample=True,
                                 pad_token_id=tokenizer.eos_token_id)

  return generated_ids


In [4]:

# prompt (no context)
intstructions_string = f""" you are a textbot that helps in finding answers to questions in the research papers, blogs,pdf's or any text context.
,make your answers more meaningful and short"

please answer the following question
"""
prompt_template_w_context = lambda context, question: f'''[INST] {intstructions_string}

{context}

Please answer to the following question. Use the context above if it is helpful.

{question}

[/INST]'''


In [6]:
import re
# Initialize an empty list to store the answers
answers = []
texts_between_inst_and_eos = []
# Iterate over each question in the dataset
for i in range(len(dataset)):
    # Get the question and the context
    question = dataset[i]["question"]
    context = dataset[i]["context"]

    # Define the prompt as the concatenation of the context and the question
    prompt = prompt_template_w_context(context, question)

    outputs = generate_response(prompt, model)

    # Decode the generated IDs and store the answer
    decoded_output = tokenizer.batch_decode(outputs)[0]
    
    answers.append(decoded_output)
    
    inst_eos_texts = re.findall(r'\[\/INST\](.*?)\<\/s\>', decoded_output, re.DOTALL)
    texts_between_inst_and_eos.extend(inst_eos_texts)


In [8]:
for text in texts_between_inst_and_eos:
    print(text.strip())

A separate standard (IS : 228-1959*) was retained for the analysis of pig iron and cast iron because a comprehensive series on chemical analysis of steels including high alloy steels was published instead (IS : 228, Parts 1 to 13), and a separate standard for the analysis of pig iron and cast iron had not been published yet.
The reproducibility for nickel content between 0.5 to 5 percent is f 0.050 percent.
The recommended concentrations of hydrochloric acid in the context are 1 : 19 and 1 : 3. The first concentration is used for diluting a concentrated hydrochloric acid solution, and the second concentration is used for the determination process in steps 6.1.2 and 6.1.3.


In [9]:

# # Print the answers
# for i, answer in enumerate(answers):
#     # print(f"Question {i+1}: {dataset[i]['question']}")
#     print("\n\n\n")
#     print(f"{answer}\n")
#     print("\n\n\n")


In [11]:
import re

# Initialize an empty list to store the texts between [/INST] and </s> tags
texts_between_inst_and_eos = []

# Iterate over each question in the dataset
for i in range(len(dataset)):
    # Get the question and the context
    question = dataset[i]["question"]
    context = dataset[i]["context"]

    # Define the prompt as the concatenation of the context and the question
    prompt = prompt_template_w_context(context, question)

    outputs = generate_response(prompt, model)

    # Decode the generated IDs
    decoded_output = tokenizer.batch_decode(outputs)[0]

    # Extract texts between [/INST] and </s> tags
    inst_eos_texts = re.findall(r'\[\/INST\](.*?)\<\/s\>', decoded_output, re.DOTALL)
    texts_between_inst_and_eos.extend(inst_eos_texts)

# Print the texts between [/INST] and </s> tags
# print("Texts between [/INST] and </s> tags:")
for text in texts_between_inst_and_eos:
    print(text.strip())


A separate standard (IS: 228-1959*) was retained for the analysis of pig iron and cast iron because a comprehensive series on chemical analysis of steels, including high alloy steels, was being published as IS: 228 (Parts 1 to 13), and a separate standard for pig iron and cast iron was not yet published.
The reproducibility for nickel content between 0.5 to 5 percent is 0.050 percent according to IS 228 (Part 5) - 1987.
The recommended concentration of hydrochloric acid for the given procedure is 1 : 19 (v/v), meaning that 1 part of concentrated hydrochloric acid should be diluted with 19 parts of water.
